Project 3 Part B: A Deep Learning Toy Model for Video Compression
===

In this project, we will use simple deep learning networks for basic video compression.

For this project, we are going to use a template that's mostly from project 2. The reason is that a video is essentially a sequence of images, in which case a network for image compression is much likely to be re-usable for video compression.

---

## 1. Objectives, Requirements and Others

### 1.1 Objectives

* Customize your model and training process for better performance -- High PSNR on test videos.
* Make sure the model complexity not too high and compression ratio not too low.

### 1.2 Requirements

* Project 3 is an **Group Project**. The report should include the team member contributions
* **Training data** can only be the data under `Image` and `Video/train` folders that provided.
* Model limit: there's a **benchmark test** at the end of this notebook. It includes model complexity test and compression ratio test. Please **test your model with this and make sure it satisfies the requirement both in complexity and compression ratio**.
* **NO pre-trained model**. You are supposed to train the network from scratch.

### 1.3 Submission

Each student is supposed to upload a **{YOUR_FIRST_NAME}_{YOUR_UNIKEY}.zip** (e.g. Sydney_haha2333.zip) file with three files:

* **a jupyternotebook** with the complete training and testing details of your final model. In other words, it is expected to reproduce the complete experiment process and reproduce the result on ```test``` datatest of your final model.
* **a .pth file** contains the trained weight of your model.
* **a PDF report** contains at least the following section:
  * Introduction
  * Method
  * Experiment & Analysis
  * Conclusion
  * (Further details are strongly encouraged, especially experiment analysis and the novelty of the method)
  * (Tables and Figures are encouraged)

### 1.4 Marking Scheme
* Code -- 25%
  * the submitted code could run without error (except the path to the dataset) -- 15%
  * the complete experimental output and log of each code cell -- 5%
  * proper comment that makes the code easy to read -- 5%

* PDF report -- 45%
  * well-written Introduction -- 5%
  * detailed discription of the method (e.g. includes an overview of the network structure of your method) -- 15%
  * detailed in-depth **quantitative** and **qualitative** analysis (e.g. tables and figures of results, how you go from baseline to the final model step-by-step and how each step affects the performance) -- 15%
  * good performance of the model -- 10%

* Presentation -- 30%
  * easy-to-follow presentation -- 15%
  * detailed in-depth **quantitative** and **qualitative** analysis -- 15%



---

## 2. Preparation for the experiments

### 2.1 Prepare the data

Please refer to the data provided in Assignment 2

**NOTE:**

1. The image data part is exactly the same training data used for project 2
2. **ONLY data in image and video/train can be used for training**, no other sources allowed.

Step-by-step

1. Put those folder (and the data inside of course) in your GoogleDrive.
2. Run the command below to mount the GoogleDrive onto Colab.
3. Check the path to the data (e.g. `'/content/drive/MyDrive/MYDATA'`), and we will use this later.

Hint: you can check the file with either bash command or use the leftmost sidebar in Colab, there's a folder shaped icon.

You can use drive.mount() to mount your Google Drive to the Colab file. For other usages, please refer to previous Ed posts and announcements.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

### 2.2 Preliminary: Conversion between images and videos

A video is essentially a sequence of consecutive images. We can convert a video to images and vice versa. Here is an example.

In [2]:
# ################################################
# import packages

import argparse
import time
import math
import random
import shutil
import sys
import glob

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms


from pathlib import Path

from PIL import Image
from torch.utils.data import Dataset


import os
import cv2
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

global_path = '/home/harryk/usyd/elec5306/imageset'

In [3]:
# convert image sequence to a video



path_to_sequence = os.path.join(global_path, 'Video/train/Bird2')
img_array = []
print('True' if os.path.exists(path_to_sequence) else 'False')
size = (0,0)

# remember to sort the images before conversion,
# otherwise they might be in wrong temporal order
for filename in sorted(glob.glob(os.path.join(path_to_sequence, '*.jpg'))):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

# use mp4v instead of DIVX
out = cv2.VideoWriter(os.path.join(path_to_sequence, 'video.mp4'),cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
# out = cv2.VideoWriter('/Users/harryk/Documents/usyd/2024sem1/ELEC5306/Project3/imageset/Video/train/Bird2/video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
print('success' if os.path.exists(os.path.join(path_to_sequence, 'video.mp4')) else 'failed')

True
success


In [4]:
# convert video to image sequence

def getFrame(vid_path, out_dir):
    if not os.path.exists(vid_path):
        print(f"Error: The video file {vid_path} does not exist.")
        return
    vidcap = cv2.VideoCapture(vid_path)
    if not vidcap.isOpened():
        print(f"Error: Could not open video stream from {vid_path}")
        return
    hasFrames = True
    count = 0
    while hasFrames:
      hasFrames,image = vidcap.read()
      if hasFrames:
        cv2.imwrite(os.path.join(out_dir, "image_"+str(count)+".jpg"), image)     # save frame as JPG file
        count += 1
      else:
        print('end of sequence')
        break

getFrame(
   os.path.join(global_path, 'Video/train/Bird2/video.mp4'),
    os.path.join(global_path, 'Video/train/Bird2/img_from_video')
)

Error: Could not open video stream from /home/harryk/usyd/elec5306/imageset/Video/train/Bird2/video.mp4


### 2.3 Get GPU ready

Change Runtime Type in the Colab menu to 'GPU'.

---

## 3. Code Template

Here is a template for training and testing a baseline model. The model is the same one we use for the Project 2. The training process are the same, while the testing are performed on video sequences.

Let's firt import packages for this project.

Then we define a Dataset class that could iterate over images under a directory.

In [5]:
# ################################################
# ImageFolder Dataset

class ImageFolder(Dataset):
    """Load an image folder database. Training and testing image samples
    are respectively stored in separate directories:

    .. code-block::

        - rootdir/
            - train/
                - img000.png
                - img001.png
                ...
            - valid/
                - img000.png
                - img001.png
                ...

    Args:
        root (string): root directory of the dataset
        transform (callable, optional): a function or transform that takes in a
            PIL image and returns a transformed version
    """

    def __init__(self, root, transform=None):
        splitdir = Path(root)

        if not splitdir.is_dir():
            raise RuntimeError(f'Invalid directory "{root}"')

        self.samples = [f for f in splitdir.iterdir() if f.is_file()]

        self.transform = transform

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            img: `PIL.Image.Image` or transformed `PIL.Image.Image`.
        """
        img = Image.open(self.samples[index]).convert("RGB")
        if self.transform:
            return self.transform(img)
        return img

    def __len__(self):
        return len(self.samples)

In [6]:
# ################################################
# SequenceFolder Dataset

class SequenceFolder(Dataset):
    """Load an image folder database. Training and testing image samples
    are respectively stored in separate directories:

    .. code-block::

        - rootdir/
            - train/
              - video 1
                - img000.png
                - img001.png
                ...
              - video 2
                - img000.png
                - img001.png
                ...
            - test/
              - video 1
                - img000.png
                - img001.png
                ...
              - video 2
                - img000.png
                - img001.png
                ...

    Args:
        root (string): root directory of the dataset
        transform (callable, optional): a function or transform that takes in a
            PIL image and returns a transformed version
        split (string): split mode ('train' or 'val')
    """

    def __init__(self, root, transform=None, split="train"):
        self.mode = split
        splitdir = Path(root) / split

        if not splitdir.is_dir():
            raise RuntimeError(f'Invalid directory "{root}"')

        self.samples = self.get_all_images(splitdir)

        self.transform = transform

    def get_all_images(self, direc):
        self.images = []
        self.image_sequence = []
        self.sequences = [f for f in direc.iterdir() if f.is_dir()]
        for sd in self.sequences:
            images = []
            for f in (sd / 'img').iterdir():
                if f.is_file():
                  images.append(f)
            self.image_sequence.append(images)
            self.images = self.images + list(sorted(images))

        return self.images

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            img: `PIL.Image.Image` or transformed `PIL.Image.Image`.
        """
        img = Image.open(self.samples[index]).convert("RGB")
        if self.transform:
            return self.transform(img)
        return img

    def __len__(self):
        return len(self.samples)

In [7]:
d = SequenceFolder(global_path)
d.image_sequence

[]

Then let's define a baseline neural network.

**NOTE:**

As in the requirement, please use the ```benchmark``` function at the end of this template to check your model's complexity and compression ratio, and see if both meet the requirement.

In [8]:
# ###################################################
# a baseline model

def conv(in_channels, out_channels, kernel_size=5, stride=2):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=kernel_size,
        stride=stride,
        padding=kernel_size // 2,
    )


def deconv(in_channels, out_channels, kernel_size=5, stride=2):
    return nn.ConvTranspose2d(
        in_channels,
        out_channels,
        kernel_size=kernel_size,
        stride=stride,
        output_padding=stride - 1,
        padding=kernel_size // 2,
    )


class Network(nn.Module):

    def __init__(self, N, M, init_weights=True, **kwargs):
        super().__init__(**kwargs)

        self.g_a = nn.Sequential(
            conv(3, N),
            conv(N, N),
            conv(N, N),
            conv(N, M),
        )

        self.g_s = nn.Sequential(
            deconv(M, N),
            deconv(N, N),
            deconv(N, N),
            deconv(N, 3),
        )

        self.N = N
        self.M = M

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        y = self.g_a(x)
        x_hat = self.g_s(y)

        return {
            "x_hat": x_hat,
        }

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def compress(self, x):
        y = self.g_a(x)
        return y

    def decompress(self, y_hat):
        x_hat = self.g_s(y_hat).clamp_(0, 1)
        return {"x_hat": x_hat}


### Network model

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, N=128, M=192):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(3, N, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(N, N * 2, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(N * 2, N * 2, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(N * 2, M, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(N)
        self.act = nn.LeakyReLU(inplace=True)

    def forward(self, x):
        x1 = self.act(self.bn1(self.conv1(x)))
        x2 = self.act(self.conv2(x1))
        x3 = self.act(self.conv3(x2))
        x4 = self.act(self.conv4(x3))
        return x4, [x1, x2, x3]

class Decoder(nn.Module):
    def __init__(self, N=128, M=192):
        super(Decoder, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(M, N * 2, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(N * 4, N * 2, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv3 = nn.ConvTranspose2d(N * 4, N, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv4 = nn.ConvTranspose2d(N * 2, 3, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.bn1 = nn.BatchNorm2d(N * 2)
        self.act = nn.LeakyReLU(inplace=True)

    def forward(self, x, skips):
        x = self.act(self.bn1(self.deconv1(x)))
        x = torch.cat((x, skips[2]), dim=1)
        x = self.act(self.deconv2(x))
        x = torch.cat((x, skips[1]), dim=1)
        x = self.act(self.deconv3(x))
        x = torch.cat((x, skips[0]), dim=1)
        x = torch.tanh(self.deconv4(x))
        return x

class CompressionNetwork(nn.Module):
    def __init__(self, N=128, M=192):
        super(CompressionNetwork, self).__init__()
        self.encoder = Encoder(N, M)
        self.decoder = Decoder(N, M)

    def forward(self, x):
        if x.dim() < 4:
            x = x.unsqueeze(0)
        y, skips = self.encoder(x)
        x_hat = self.decoder(y, skips)
        return {"x_hat": x_hat}

    def compress(self, x):
        y, _ = self.encoder(x)
        return y

    def decompress(self, y):
        x_hat = self.decoder(y, [None, None, None]).clamp_(0, 1)
        return {"x_hat": x_hat}


Here are some other ingredients for training and testing.

In [10]:



class Loss(nn.Module):

    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, output, target):
        out = {}
        out["mse_loss"] = self.mse(output["x_hat"], target)
        out["loss"] = out["mse_loss"] * 255

        return out

In [11]:
class AverageMeter:
    """Compute running average."""

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
def configure_optimizers(net, learning_rate):

    optimizer = optim.Adam(
        net.parameters(),
        lr=learning_rate,
    )

    return optimizer

In [13]:
def save_checkpoint(state, is_best, filename="checkpoint.pth.tar"):
    torch.save(state, filename)
    if is_best:
        print('Saving best model...')
        shutil.copyfile(filename, "checkpoint_best_loss.pth.tar")

In [14]:
def PSNR(img1, img2):
    # img1 and img2 within range [0, 1]
    # img1 shape: (B, C, H, W)
    # img2 shape: (B, C, H, W)

    img1, img2 = img1.detach(), img2.detach()
    img1 = img1 * 255
    img2 = img2 * 255
    batch_size = img1.shape[0]
    img1 = img1.reshape(batch_size, -1)
    img2 = img2.reshape(batch_size, -1)
    mse = torch.mean((img1 - img2) ** 2)
    return torch.mean(20 * torch.log10(255.0 / torch.sqrt(mse)))

After we almost got everything we need, we define a `train_one_epoch` function for training the model for one epoch.

In [15]:
def train_one_epoch(
    model, criterion, train_dataloader, optimizer, epoch, clip_max_norm
):
    model.train()
    device = next(model.parameters()).device

    for i, d in enumerate(train_dataloader):
        d = d.to(device)

        optimizer.zero_grad()

        out_net = model(d)

        out_criterion = criterion(out_net, d)
        out_criterion["loss"].backward()
        if clip_max_norm > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_max_norm)
        optimizer.step()

        if i % 20 == 0:
            print(
                f"Train epoch {epoch}: ["
                f"{i*len(d)}/{len(train_dataloader.dataset)}"
                f" ({100. * i / len(train_dataloader):.0f}%)]"
                f'\tLoss: {out_criterion["loss"].item():.3f} |'
                f'\tMSE loss: {out_criterion["mse_loss"].item():.3f}'
            )


Likewise we further define a `test_epoch` function.

Unlike project 2, this time the metric is measured on videos instead of images.

In [16]:
def test_epoch(epoch, test_dataset, transform, model, criterion):
    model.eval()
    device = next(model.parameters()).device

    mse_loss = AverageMeter()
    psnr = AverageMeter()

    with torch.no_grad():
        for i, video in enumerate(test_dataset.image_sequence):
            psnr_video = AverageMeter()
            for j, image in enumerate(video):
                image = Image.open(image).convert("RGB")
                image = transform(image)
                d = image.to(device).unsqueeze(0)
                out_net = model(d)
                d_out = out_net['x_hat']
                out_criterion = criterion(out_net, d)
                psnr_video.update(PSNR(d, d_out))

            mse_loss.update(out_criterion["mse_loss"])
            psnr.update(psnr_video.avg)

    print(
        f"Test epoch {epoch}: Average losses:"
        f"\tMSE loss: {mse_loss.avg:.3f}"
        f'\tSequence-wise PSNR: {psnr.avg: .3f}\n'
    )

    return mse_loss.avg

def test_epoch(epoch, test_dataset, model, criterion, batch_size=256):
    model.eval()
    device = next(model.parameters()).device

    mse_loss = AverageMeter()
    psnr = AverageMeter()

    data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    with torch.no_grad():
        for batch in data_loader:
            psnr_video = AverageMeter()
            for images in batch:
                images = images.to(device)
                out_net = model(images)
                d_out = out_net['x_hat']
                out_criterion = criterion(out_net, images)
                psnr_video.update(PSNR(images, d_out))

            mse_loss.update(out_criterion["mse_loss"])
            psnr.update(psnr_video.avg)

    print(
        f"Test epoch {epoch}: Average losses:"
        f"\tMSE loss: {mse_loss.avg:.3f}"
        f'\tSequence-wise PSNR: {psnr.avg: .3f}\n'
    )

    return mse_loss.avg

Finally we define a main function, and complete the whole training/testing process.

At the start of this main function, we define some arguments that's useful for training.

In [17]:
# !ls /Your-path/ELEC5306

Define some arguments useful for training/validation.

In [18]:
seed = 123                                        # for reproducibility
cuda = True                                       # use GPU
save = True                                       # save trained model
image_dataset = os.path.join(global_path,'train')  # path to the root of the image dataset
sequence_dataset = os.path.join(global_path,'Video')  # path to the root of the video dataset
checkpoint = ''                                   # load pretrained model
epochs = 50                                       # total training epochs
clip_max_norm = 1.0                               # avoid gradient explosion
patch_size = (96, 128)                           # input size for the training network
learning_rate = 3e-4
batch_size = 32
test_batch_size = 32
num_workers = 256                         # multi-process for loading training data
N = 128
M = 192

### Train

In [19]:
torch.manual_seed(seed)
random.seed(seed)

train_transforms = transforms.Compose(
    [transforms.RandomCrop(patch_size), transforms.ToTensor()]
)

test_transforms = transforms.Compose(
    [transforms.CenterCrop(patch_size), transforms.ToTensor()]
)

# train_dataset = ImageFolder(image_dataset, transform=train_transforms)
train_dataset = SequenceFolder(sequence_dataset, split="train", transform=train_transforms)
test_dataset = SequenceFolder(sequence_dataset, split="test", transform=test_transforms)

device = "cuda" if cuda and torch.cuda.is_available() else "mps"

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
    pin_memory=(device == "cuda" or "mps"),
)





net = CompressionNetwork(N, M)
net = net.to(device)

optimizer = configure_optimizers(net, learning_rate)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min",factor=0.5,patience=10)
criterion = Loss()

last_epoch = 0
if checkpoint:  # load from previous checkpoint
    print("Loading", checkpoint)
    checkpoint = torch.load(checkpoint, map_location=device)
    last_epoch = checkpoint["epoch"] + 1
    net.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    lr_scheduler.load_state_dict(checkpoint["lr_scheduler"])

best_loss = float("inf")
train_time = AverageMeter()
for epoch in range(last_epoch, epochs):
    print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
    epoch_train_start = time.time()
    train_one_epoch(
        net,
        criterion,
        train_dataloader,
        optimizer,
        epoch,
        clip_max_norm,
    )
    epoch_train_end = time.time()
    train_time.update(epoch_train_end - epoch_train_start)
    loss = test_epoch(epoch, test_dataset, net, criterion)
    lr_scheduler.step(loss)

    is_best = loss < best_loss
    best_loss = min(loss, best_loss)

    if save:
      save_checkpoint(
          {
              "epoch": epoch,
              "state_dict": net.state_dict(),
              "loss": loss,
              "optimizer": optimizer.state_dict(),
              "lr_scheduler": lr_scheduler.state_dict(),
          },
          is_best,
      )
print('the overall training time (exclude testing) is {} min'.format(train_time.sum / 60))

/home/harryk/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:556: UserWarning: This DataLoader will create 256 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Learning rate: 0.0003
Train epoch 0: [0/13590 (0%)]	Loss: 234.625 |	MSE loss: 0.920
Train epoch 0: [640/13590 (5%)]	Loss: 29.266 |	MSE loss: 0.115
Train epoch 0: [1280/13590 (9%)]	Loss: 13.376 |	MSE loss: 0.052
Train epoch 0: [1920/13590 (14%)]	Loss: 9.596 |	MSE loss: 0.038
Train epoch 0: [2560/13590 (19%)]	Loss: 8.582 |	MSE loss: 0.034
Train epoch 0: [3200/13590 (24%)]	Loss: 8.088 |	MSE loss: 0.032
Train epoch 0: [3840/13590 (28%)]	Loss: 4.302 |	MSE loss: 0.017
Train epoch 0: [4480/13590 (33%)]	Loss: 4.095 |	MSE loss: 0.016
Train epoch 0: [5120/13590 (38%)]	Loss: 3.253 |	MSE loss: 0.013
Train epoch 0: [5760/13590 (42%)]	Loss: 2.899 |	MSE loss: 0.011
Train epoch 0: [6400/13590 (47%)]	Loss: 3.835 |	MSE loss: 0.015
Train epoch 0: [7040/13590 (52%)]	Loss: 2.352 |	MSE loss: 0.009
Train epoch 0: [7680/13590 (56%)]	Loss: 1.760 |	MSE loss: 0.007
Train epoch 0: [8320/13590 (61%)]	Loss: 3.437 |	MSE loss: 0.013
Train epoch 0: [8960/13590 (66%)]	Loss: 2.014 |	MSE loss: 0.008
Train epoch 0: [9600/

/home/harryk/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3, 96, 128])) that is different to the input size (torch.Size([1, 3, 96, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test epoch 0: Average losses:	MSE loss: 0.008	Sequence-wise PSNR:  21.561

Saving best model...
Learning rate: 0.0003
Train epoch 1: [0/13590 (0%)]	Loss: 1.549 |	MSE loss: 0.006
Train epoch 1: [640/13590 (5%)]	Loss: 1.199 |	MSE loss: 0.005
Train epoch 1: [1280/13590 (9%)]	Loss: 1.251 |	MSE loss: 0.005
Train epoch 1: [1920/13590 (14%)]	Loss: 1.454 |	MSE loss: 0.006
Train epoch 1: [2560/13590 (19%)]	Loss: 1.021 |	MSE loss: 0.004
Train epoch 1: [3200/13590 (24%)]	Loss: 2.222 |	MSE loss: 0.009
Train epoch 1: [3840/13590 (28%)]	Loss: 1.322 |	MSE loss: 0.005
Train epoch 1: [4480/13590 (33%)]	Loss: 1.514 |	MSE loss: 0.006
Train epoch 1: [5120/13590 (38%)]	Loss: 1.258 |	MSE loss: 0.005
Train epoch 1: [5760/13590 (42%)]	Loss: 1.871 |	MSE loss: 0.007
Train epoch 1: [6400/13590 (47%)]	Loss: 1.610 |	MSE loss: 0.006
Train epoch 1: [7040/13590 (52%)]	Loss: 1.523 |	MSE loss: 0.006
Train epoch 1: [7680/13590 (56%)]	Loss: 0.581 |	MSE loss: 0.002
Train epoch 1: [8320/13590 (61%)]	Loss: 2.593 |	MSE loss:

# benchmark

### Benchmark Test (VERY IMPORTANT)

1. Please use 'torch.save(model.state_dict(), PATH)' to save your trained model

2. Please use the below code block, to instantiate your model as object 'test_model' for marking. We will use 'load_state_dict()' to load your model. Please make sure your saved model can pass the loading test. You can use the below code block to test.

3. Please include your trained model weight in the submission.

4. **If we cannot load your uploaded weight, you will get zero mark for this assignment.**

In [ ]:
### EDIT REQUIRED !!! ###

### Please put the structure definition of your model here

PATH = 'checkpoint_best_loss.pth.tar'
test_model = Network(N,M)
# test_model.load_state_dict(torch.load(PATH)['state_dict'])
test_model.eval()

Network(
  (g_a): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (2): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (3): Conv2d(128, 192, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  )
  (g_s): Sequential(
    (0): ConvTranspose2d(192, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (1): ConvTranspose2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (2): ConvTranspose2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (3): ConvTranspose2d(128, 3, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  )
)

### *(Do not modify any of the code below this section)*

**Make sure your model has ``compress`` method like the one in the example network**

please make sure your model can get through this benchmark, in which case it prints ``Final test result of your model``.

Otherwise refer to the output for the guidance to adjust your network.

In [ ]:
!pip install pthflops

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# ################################################
# import packages

import argparse
import time
import math
import random
import shutil
import sys
import glob

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms


from pathlib import Path

from PIL import Image
from torch.utils.data import Dataset

In [ ]:
seed = 123                                        # for reproducibility
cuda = True                                       # use GPU
save = True                                       # save trained model
image_dataset = os.path.join(global_path,'train')  # path to the root of the image dataset
sequence_dataset = os.path.join(global_path,'Video')  # path to the root of the video dataset
checkpoint = ''                                   # load pretrained model
epochs = 10                                       # total training epochs
clip_max_norm = 1.0                               # avoid gradient explosion
patch_size = (256, 256)                           # input size for the training network
learning_rate = 1e-4
batch_size = 16
test_batch_size = 16
num_workers = batch_size                          # multi-process for loading training data
N = 128
M = 192

# ################################################
# SequenceFolder Dataset

class SequenceFolder(Dataset):
    """Load an image folder database. Training and testing image samples
    are respectively stored in separate directories:

    .. code-block::

        - rootdir/
            - train/
              - video 1
                - img000.png
                - img001.png
                ...
              - video 2
                - img000.png
                - img001.png
                ...
            - test/
              - video 1
                - img000.png
                - img001.png
                ...
              - video 2
                - img000.png
                - img001.png
                ...

    Args:
        root (string): root directory of the dataset
        transform (callable, optional): a function or transform that takes in a
            PIL image and returns a transformed version
        split (string): split mode ('train' or 'val')
    """

    def __init__(self, root, transform=None, split="train"):
        self.mode = split
        splitdir = Path(root) / split

        if not splitdir.is_dir():
            raise RuntimeError(f'Invalid directory "{root}"')

        self.samples = self.get_all_images(splitdir)

        self.transform = transform

    def get_all_images(self, direc):
        self.images = []
        self.image_sequence = []
        self.sequences = [f for f in direc.iterdir() if f.is_dir()]
        for sd in self.sequences:
            images = []
            for f in (sd / 'img').iterdir():
                if f.is_file():
                  images.append(f)
            self.image_sequence.append(images)
            self.images = self.images + list(sorted(images))

        return self.images

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            img: `PIL.Image.Image` or transformed `PIL.Image.Image`.
        """
        img = Image.open(self.samples[index]).convert("RGB")
        if self.transform:
            return self.transform(img)
        return img

    def __len__(self):
        return len(self.samples)

test_dataset = SequenceFolder(sequence_dataset, split="test", transform=None)

In [ ]:




class AverageMeter:
    """Compute running average."""

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def PSNR(img1, img2):
    # img1 and img2 within range [0, 1]
    # img1 shape: (B, C, H, W)
    # img2 shape: (B, C, H, W)

    img1, img2 = img1.detach(), img2.detach()
    img1 = img1 * 255
    img2 = img2 * 255
    batch_size = img1.shape[0]
    img1 = img1.reshape(batch_size, -1)
    img2 = img2.reshape(batch_size, -1)
    mse = torch.mean((img1 - img2) ** 2)
    return torch.mean(20 * torch.log10(255.0 / torch.sqrt(mse)))

def final_test_epoch(test_dataset, transform, model, criterion):
    model.eval()
    device = next(model.parameters()).device

    mse_loss = AverageMeter()
    psnr = AverageMeter()

    with torch.no_grad():
        for video in test_dataset.image_sequence:
            psnr_video = AverageMeter()
            for image in video:
                image = Image.open(image).convert("RGB")
                image = transform(image)
                d = image.to(device).unsqueeze(0)
                out_net = model(d)
                d_out = out_net['x_hat']
                out_criterion = criterion(out_net, d)
                psnr_video.update(PSNR(d, d_out))

            mse_loss.update(out_criterion["mse_loss"])
            psnr.update(psnr_video.avg)

    print(
        f"Final test of model: Average losses:"
        f"\tMSE loss: {mse_loss.avg:.3f}"
        f'\tSequence-wise PSNR: {psnr.avg: .3f}\n'
    )

    return mse_loss.avg

class Loss(nn.Module):

    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, output, target):
        out = {}
        out["mse_loss"] = self.mse(output["x_hat"], target)
        out["loss"] = out["mse_loss"] * 255

        return out

test_transforms = transforms.Compose(
    [transforms.CenterCrop(patch_size), transforms.ToTensor()]
)

criterion = Loss()

In [ ]:
from pthflops import count_ops

MAX_GFLOPS = 3000
MIN_RATIO = 0.7

def benchmark(yournet):
    yournet = yournet.cuda()
    x = torch.randn((1, 3, 500, 500)).cuda()
    try:
      yournet.compress(x)
    except AttributeError:
      print('does your network have a ```def compress(self, input)``` function?')
      print('refer to the baseline model in the template')
      return

    est, _ = count_ops(yournet, x, print_readable=False, verbose=False)
    est = int(est / 1e9)
    if est <= MAX_GFLOPS:
      print('#' * 30)
      print('[Acceptable Model Complexity]')
      print('#' * 30)
      print('\n')
    else:
      assert 0, 'Your model complexity is {} GFLOPS, the acceptable maximum is {} GFLOPS. Make your model smaller'.format(est, MAX_GFLOPS)

    compressed = yournet.compress(x)
    ratio = 1 - compressed.numel() / x.numel()
    if ratio >= MIN_RATIO:
      print('#' * 30)
      print('[Acceptable Compression Ratio]')
      print('#' * 30)
    else:
      assert 0, 'Current compression ratio is {} , the acceptable lowest ratio is {}. Make it higher'.format(ratio, MIN_RATIO)

    print('\n')

    print('#' * 30)
    print('Final test result of your model')
    print('#' * 30)
    print('\n')

    loss = final_test_epoch(test_dataset, test_transforms, yournet, criterion)

Use the below code if you need to upload the model weight file from your local PC.

In [ ]:
# from google.colab import files

# model_weight = files.upload()
# filename = next(iter(model_weight))
# print("Uploaded " + filename)

In [ ]:
# an EXAMPLE network to run the test
# test_model = Network(N, M)
# test_model.load_state_dict(torch.load('checkpoint_best_loss.pth.tar')['state_dict'])
# test_model.eval()
benchmark(test_model)

##############################
[Acceptable Model Complexity]
##############################


##############################
[Acceptable Compression Ratio]
##############################


##############################
Final test result of your model
##############################


Final test of model: Average losses:	MSE loss: 17.765	Sequence-wise PSNR: -11.255

